In [ ]:
'''
This is the notebook for cleaning weather data in commercial buildings
'''

In [ ]:
import pandas as pd 
import numpy as np 
import os 
import csv

In [ ]:
final_path = "Final DB/commercial/"
raw_path = "Raw DB/commercial/Energy"

In [ ]:
def clean_file_weather(file):
    """
    Delete unneccessary weather features
    """
    if not os.path.isfile(file):
        print("File does not exit")
        return None 
    else:
        columns = ["Max_temp","Min_temp","Dewpoint","Humidity","Max_windspeed","Min_windspeed","Max_pressure","Min_pressure","Precipitation"]
        df = pd.read_csv(file)
        # consider time, temp max, temp min, dew avg, avg humid, wind max, wind min, pressure max, pressure min, precipitation
        df = df.drop(["Time","Avgtemp","Maxdew","Mindew","Maxhumid","Minhumid","Avgwind","Avgpressure"],axis=1)
        # check leap year, and insert 29.02 row
        if df.shape[0] == 365:
            zeros = np.zeros((df.shape[1],))
            df = df.values
            df = np.insert(df,(31+28)*9,zeros).reshape(366,9)
            # convert back to DataFrame
            df = pd.DataFrame(df,columns=columns)
        else:
            df.columns = columns
        return df

In [ ]:
def merge_files_weather(file_list,file_name):
     """
     Merge four-year weather into one by averaging
     """
    columns = ["Max_temp","Min_temp","Dewpoint","Humidity","Max_windspeed","Min_windspeed","Max_pressure","Min_pressure","Precipitation"]
    # create temp list to hold all values of dfs
    df = np.zeros((366,9))
    for file in file_list:
         _temp = clean_file(file).values
         #sum up
         df += _temp
    # take average for 4 years
    df /= 4.0
    # take care 29.02
    df[31+28] *= 4.0
    # convert to df
    df = pd.DataFrame(df,columns=columns)
    # add time
    
    dates = pd.date_range("20040101",periods = 366)
    df["Time"] = dates
    # rearrange columns order
    
    columns = df.columns.to_list()
    columns = columns[-1:] + columns[:-1]
    df = df[columns]
    # write file
    df.to_csv(r"".join(file_name), index = True, header = True)
    # return 
    return df

In [ ]:
file_list = ["2016.csv","2017.csv","2018.csv","2019.csv"]
merge_files(file_list,final_path + "merged_weather.csv").head(60)